## Model Input Data Pre-Processing
----------------------------------

### 1. Import packages

In [22]:
#!conda install -c https://conda.anaconda.org/plotly plotly -y
#!pip install pydot
#!pip install graphviz
#!conda install -c anaconda py-xgboost -y
#!pip install shap
#!pip install dill
#!pip install matplotlib
#!pip install seaborn
#!pip install missingpy
#!pip3 install pickle5

In [1]:
import pandas as pd
import sqlite3 as sql
import numpy as np
from IPython.display import HTML
from datetime import datetime
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()  # set plot style
from collections import Counter
import scipy as sp
import sklearn as sk
import sys
import plotly.express as px
import pydot
import pickle5 as pickle
import time
import xgboost as xgb
import shap
import dill
import gc
import subprocess

In [2]:
import os
from pandas.plotting import scatter_matrix
from matplotlib import pyplot

from collections import defaultdict

from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.tree import export_graphviz
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

from sklearn.model_selection import KFold
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

### 2. Read in csv files

In [25]:
diags = pd.read_csv(r'/Users/amomenzadeh/Desktop/DM_csv/i2b2diagnosis_codes.csv')

In [5]:
demos = pd.read_csv(r'/Users/amomenzadeh/Desktop/DM_csv/i2b2demographics.csv')

In [6]:
vitals = pd.read_csv(r'/Users/amomenzadeh/Desktop/DM_csv/i2b2vitals.csv')

In [31]:
social = pd.read_csv(r'/Users/amomenzadeh/Desktop/DM_csv/i2b2social_history_lifestyle.csv',low_memory=False)

In [10]:
labs = pd.read_csv(r'/Users/amomenzadeh/Desktop/DM_csv/i2b2diagnostic_results.csv')

In [26]:
icd9map= pd.read_csv(r'/Users/amomenzadeh/Desktop/DM_csv/phecode_icd9_rolled.csv',index_col=0)

In [27]:
icd10map=pd.read_csv(r'/Users/amomenzadeh/Desktop/DM_csv/phecode_icd10.csv',index_col=0)

### 3. Clean Diagnosis input data

#### A. Complication Lists

Compile all children codes for each complication

In [18]:
## Ocular disease codes (H35, 362, E11.3, 250.50, 366.41, 365)
opthal_comps_list = ['H35.0', 'H35.00', 'H35.01', 'H35.011', 'H35.012', 'H35.013', \
                     'H35.019', 'H35.02', 'H35.021', 'H35.022', 'H35.023', 'H35.029', \
                     'H35.03', 'H35.031', 'H35.032', 'H35.033', 'H35.039', 'H35.04', \
                      'H35.041', 'H35.042', 'H35.043', 'H35.049', 'H35.05', 'H35.051', \
                      'H35.052', 'H35.053', 'H35.059', 'H35.06', 'H35.061', 'H35.062', \
                      'H35.063', 'H35.069', 'H35.07', 'H35.071', 'H35.072', 'H35.073', \
                      'H35.079', 'H35.09', 'H35.1', 'H35.10', 'H35.101', 'H35.102', 'H35.103', \
                      'H35.109', 'H35.11', 'H35.111', 'H35.112', 'H35.113', 'H35.119', 'H35.12', \
                      'H35.121', 'H35.122', 'H35.123', 'H35.129', 'H35.13', 'H35.131', 'H35.132', \
                      'H35.133', 'H35.139', 'H35.14', 'H35.141', 'H35.142', 'H35.143', \
                      'H35.149', 'H35.15', 'H35.151', 'H35.152', 'H35.153', 'H35.159', 'H35.16', \
                      'H35.161', 'H35.162', 'H35.163', 'H35.169', 'H35.17', 'H35.171', 'H35.172', \
                      'H35.173', 'H35.179', 'H35.2', 'H35.20', 'H35.21', 'H35.22', 'H35.23', \
                      'H35.3', 'H35.30', 'H35.31', 'H35.311', 'H35.3110', 'H35.3111', 'H35.3112', \
                      'H35.3113', 'H35.3114', 'H35.312', 'H35.3120', 'H35.3121', 'H35.3122', \
                      'H35.3123', 'H35.3124', 'H35.313', 'H35.3130', 'H35.3131', 'H35.3132', \
                      'H35.3133', 'H35.3134', 'H35.319', 'H35.3190', 'H35.3191', 'H35.3192', \
                      'H35.3193', 'H35.3194', 'H35.32', 'H35.321', 'H35.3210', 'H35.3211', \
                      'H35.3212', 'H35.3213', 'H35.322', 'H35.3220', 'H35.3221', 'H35.3222', \
                      'H35.3223', 'H35.323', 'H35.3230', 'H35.3231', 'H35.3232', 'H35.3233', \
                      'H35.329', 'H35.3290', 'H35.3291', 'H35.3292', 'H35.3293', 'H35.33', 'H35.34', \
                      'H35.341', 'H35.342', 'H35.343', 'H35.349', 'H35.35', 'H35.351', 'H35.352', \
                      'H35.353', 'H35.359', 'H35.36', 'H35.361', 'H35.362', 'H35.363', 'H35.369', \
                      'H35.37', 'H35.371', 'H35.372', 'H35.373', 'H35.379', 'H35.38', 'H35.381', \
                      'H35.382', 'H35.383', 'H35.389', 'H35.4', 'H35.40', 'H35.41', 'H35.411', \
                      'H35.412', 'H35.413', 'H35.419', 'H35.42', 'H35.421', 'H35.422', 'H35.423', \
                      'H35.429', 'H35.43', 'H35.431', 'H35.432', 'H35.433', 'H35.439', 'H35.44', \
                      'H35.441', 'H35.442', 'H35.443', 'H35.449', 'H35.45', 'H35.451', 'H35.452', \
                      'H35.453', 'H35.459', 'H35.46', 'H35.461', 'H35.462', 'H35.463', 'H35.469', \
                      'H35.5', 'H35.50', 'H35.51', 'H35.52', 'H35.53', 'H35.54', 'H35.6', 'H35.60', \
                      'H35.61', 'H35.62', 'H35.63', 'H35.7', 'H35.70', 'H35.71', 'H35.711', \
                      'H35.712', 'H35.713', 'H35.719', 'H35.72', 'H35.721', 'H35.722', 'H35.723', \
                      'H35.729', 'H35.73', 'H35.731', 'H35.732', 'H35.733', 'H35.739', 'H35.8', \
                      'H35.81', 'H35.82', 'H35.89', 'H35.9', '362.0', '362.01', '362.02', '362.03', '362.04', '362.05', '362.06', '362.07', \
                      '362.1', '362.10', '362.11', '362.12', '362.13', '362.14', '362.15', '362.16', '362.17', \
                      '362.18', '362.2', '362.20', '362.21', '362.22', '362.23', '362.24', '362.25', '362.26', \
                      '362.27', '362.29', '362.3', '362.30', '362.31', '362.32', '362.33', '362.34', '362.35', \
                      '362.36', '362.37', '362.40', '362.4', '362.41', '362.42', '362.43', '362.50', '362.5', \
                      '362.51', '362.52', '362.53', '362.54', '362.55', '362.56', '362.57', '362.60', '362.6', \
                      '362.61', '362.62', '362.63', '362.64', '362.65', '362.66', '362.70', '362.7', '362.71', \
                      '362.72', '362.73', '362.74', '362.75', '362.76', '362.77', '362.80', '362.81', '362.82', \
                      '362.83', '362.84', '362.85', '362.89', '362.9', 'E11.30', 'E11.3', 'E11.31', 'E11.311', 'E11.319', \
                      'E11.32', 'E11.321', 'E11.3211', 'E11.3212', 'E11.3213', 'E11.3219', 'E11.329', 'E11.3291', \
                      'E11.3292', 'E11.3293', 'E11.3299', 'E11.33', 'E11.331', 'E11.3311', 'E11.3312', 'E11.3313', \
                      'E11.3319', 'E11.339', 'E11.3391', 'E11.3392', 'E11.3393', 'E11.3399', 'E11.34', 'E11.341', \
                      'E11.3411', 'E11.3412', 'E11.3413', 'E11.3419', 'E11.349', 'E11.3491', 'E11.3492', 'E11.3493', \
                      'E11.3499', 'E11.35', 'E11.351', 'E11.3511', 'E11.3512', 'E11.3513', 'E11.3519', 'E11.352', \
                      'E11.3521', 'E11.3522', 'E11.3523', 'E11.3529', 'E11.353', 'E11.3531', 'E11.3532', 'E11.3533', \
                      'E11.3539', 'E11.354', 'E11.3541', 'E11.3542', 'E11.3543', 'E11.3549', 'E11.355', 'E11.3551', \
                      'E11.3552', 'E11.3553', 'E11.3559', 'E11.359', 'E11.3591', 'E11.3592', 'E11.3593', 'E11.3599', \
                      'E11.36', 'E11.37', 'E11.37X1', 'E11.37X2', 'E11.37X3', 'E11.37X9', 'E11.39', '250.50', '366.41' \
                      '365', '365.0', '365.00', '365.01', '365.02', '365.03', '365.04', '365.05', '365.06', '365.1', '365.10', \
                      '365.11', '365.12', '365.13', '365.14', '365.15', '365.2', '365.20', '365.21', '365.22', '365.23', '365.24', '365.3',\
                      '365.30', '365.31', '365.32', '365.4', '365.40', '365.41', '365.42', '365.43', '365.44', '365.5','365.50', '365.51', '365.52', '365.59',\
                      '365.60', '365.6', '365.61', '365.62', '365.63', '365.64', '365.65', '365.7', '365.70', '365.71', '365.72', '365.73',\
                      '365.74', '365.8', '365.80', '365.81', '365.82', '365.83', '365.89', '365.9', '365.90'
]

In [19]:
## Kidney disease codes (250.40, 403, 404, 581, 583, 584, 585, 586, 
## 588, 593, E11.2, I12, I13, N04, N05, N08, N17, N18, N19, N25, N29)
kidney_comps_list = ['250.40', '403', '403.0', '403.00', '403.01', '403.1', '403.10', '403.11', '403.9', '403.90', '403.91'\
                     '404', '404.0', '404.00', '404.01', '404.02', '404.03', '404.1', '404.10', '404.11', '404.12', '404.13', 
                     '404.9', '404.90', '404.91', '404.92', '404.93', '581', '581.0', '581.1', '581.2', '581.3', '581.8', \
                     '581.81', '581.89', '581.9', '583', '583.0', '583.1', '583.2', '583.4', '583.6', '583.7', '583.8', \
                     '583.81', '583.89', '583.9', '584', '584.5', '584.6', '584.7', '584.8', '584.9', \
                     '585', '585.1', '585.2', '585.3', '585.4', '585.5', '585.6', '585.9', '586', '588', '588.0', '588.1',\
                     '588.8', '588.81', '588.89', '588.9', '593', '593.0', '593.1', '593.2', '593.3', '593.4', '593.5', \
                     '593.6', '593.7', '593.70', '593.71', '593.72', '593.73', '593.8', '593.81', '593.82', '593.89', '593.9',\
                     'E11.2', 'E11.21', 'E11.22', 'E11.29', 'I12', 'I12.0', 'I12.9', 'I13', 'I13.0', 'I13.1', 'I13.10', 'I13.11',\
                     'I13.2', 'N04', 'N04.0', 'N04.1', 'N04.2', 'N04.3', 'N04.4', 'N04.5', 'N04.6', 'N04.7', 'N04.8', 'N04.9', \
                     'N04.A', 'N05.0', 'N05.1', 'N05.2', 'N05.3', 'N05.4', 'N05.5', 'N05.6', 'N05.7', 'N05.8', 'N05.9', 'N08', \
                     'N17.0', 'N17.1', 'N17.2', 'N17.8', 'N17.9', 'N18.1', 'N18.2', 'N18.3', 'N18.4', 'N18.5', 'N18.6', 'N18.9',\
                     'N19', 'N25.0', 'N25.1', 'N25.8', 'N25.81', 'N25.89', 'N25.9', 'N29'
]

In [20]:
## Cardiovascular Disease (250.70, 410, 412, 413, 414, 428, E11.5, I20, I21, I25, I50)
cv_comps_list = ['250.70', '410', '410.0', '410.00', '410.01', '410.02', '410.1', '410.10', '410.11', '410.12', '410.2', '410.20', \
                 '410.21', '410.22', '410.3', '410.30', '410.31', '410.32', '410.4', '410.40', '410.41', '410.42', '410.5', '410.50', \
                 '410.51', '410.52', '410.6', '410.60', '410.61', '410.62', '410.7', '410.70', '410.71', '410.72', '410.8', '410.80', \
                 '410.81', '410.82', '410.9', '410.90', '410.91', '410.92', '412', '413', '413.0', '413.1', '413.9', '414', '414.0', \
                 '414.00', '414.01', '414.02', '414.03', '414.04', '414.05', '414.06', '414.07', '414.1', '414.10', '414.11', '414.12',\
                 '414.19', '414.2', '414.3', '414.4', '414.8', '414.9', '428', '428.0', '428.1', '428.2', '428.20', '428.21', '428.22', \
                 '428.23', '428.3', '428.30', '428.31', '428.32', '428.33', '428.4', '428.40', '428.41', '428.42', '428.43', '428.9', \
                 'E11.5', 'E11.51', 'E11.52', 'E11.59', 'I20', 'I20.0', 'I20.1', 'I20.8', 'I20.9', 'I21', 'I21.0', 'I21.01', 'I21.02', \
                 'I21.09', 'I21.1', 'I21.11', 'I21.19', 'I21.2', 'I21.21', 'I21.29', 'I21.3', 'I21.4', 'I21.9', 'I21.A', 'I21.A1', 'I21.A9',\
                 'I25', 'I25.1', 'I25.10', 'I25.11', 'I25.110', 'I25.111', 'I25.118', 'I25.119', 'I25.2', 'I25.3', 'I25.4', 'I25.41', \
                 'I25.42', 'I25.5', 'I25.6', 'I25.7', 'I25.70', 'I25.700', 'I25.701', 'I25.708', 'I25.709', 'I25.71', 'I25.710', 'I25.711',\
                 'I25.718', 'I25.719', 'I25.72', 'I25.720', 'I25.721', 'I25.728', 'I25.729', 'I25.73', 'I25.730', 'I25.731', 'I25.738', \
                 'I25.739', 'I25.75', 'I25.750', 'I25.751', 'I25.758', 'I25.759', 'I25.76', 'I25.760', 'I25.761', 'I25.768', 'I25.769', \
                 'I25.79', 'I25.790', 'I25.791', 'I25.798', 'I25.799', 'I25.8', 'I25.81', 'I25.82', 'I25.83', 'I25.84', 'I25.89', 'I25.9',\
                 'I50', 'I50.1', 'I50.2', 'I50.20', 'I50.21', 'I50.22', 'I50.23', 'I50.3', 'I50.30', 'I50.31', 'I50.32', 'I50.33', 'I50.4',\
                 'I50.40', 'I50.41', 'I50.42', 'I50.43', 'I50.8', 'I50.81', 'I50.810', 'I50.811', 'I50.812', 'I50.813', 'I50.814', 'I50.82',\
                 'I50.83', 'I50.84', 'I50.89', 'I50.9'
]

In [24]:
## Neuropathies (250.60, 337.1, 353.5, 354.8, 354.9, 355.7, 355.8, 357.2, E11.4, G62.9, G63)
neurop_comps_list = ['250.60', '337.1', '353.5', '354.8', '354.9', '355.7', '355.71', '355.79', '355.8', '357.2', 'E11.4', 'E11.40',\
                     'E11.41', 'E11.42', 'E11.43', 'E11.44', 'E11.49', 'G62.9', 'G63', 'G63.2', 'G63.3'
]

In [13]:
comp_names = ['OPTH', 'CVD', 'KIDNEY', 'NEUROPATH']

#### B. Diagnoses

In [621]:
len(diags.patient_num.unique())

30854

In [28]:
# remove hours/minutes/seconds places from the date column (leaving only Y-m-d)
temp1 = diags["dx_date_shifted"].tolist()
temp2 = []
for value in temp1:
    x = value.split(" ")
    temp2.append(x[0])
diags["dx_date_shifted"] = temp2

In [29]:
diags.dx_date_shifted = pd.to_datetime(diags.dx_date_shifted, format = '%Y-%m-%d') # convert dx date column to datetime

In [30]:
# list of columns to drop
col_drop_list_diags = ['enc_type', 'provider_id', 'provider_name', 'provider_title', \
                       'dx_source', 'dx_origin', 'raw_pdx', 'sourcesystem_cd', 'encounter_num', \
                       'dx_name', 'pdx', 'dx_type']
diags.drop(col_drop_list_diags, axis=1, inplace=True) # drop columns from df
diags.rename({'dx_date_shifted':'dx_date'}, axis='columns', inplace=True) # rename the diagnosis date column
diags.sort_values(by='dx_date', inplace=True) # sort df by date

In [31]:
# Grouping complication codes together
diags.dx_code.replace(to_replace=opthal_comps_list, value='OPTH', inplace=True)
diags.dx_code.replace(to_replace=cv_comps_list, value='CVD', inplace=True)
diags.dx_code.replace(to_replace=kidney_comps_list, value='KIDNEY', inplace=True)
diags.dx_code.replace(to_replace=neurop_comps_list, value='NEUROPATH', inplace=True)

In [33]:
diags_grouped = diags.copy()

In [34]:
diags_grouped['edited_code'] = ''
diags_grouped['edited_code1'] = ''
def edit_codes(df):
    df.edited_code = (df.dx_code.str.split('.').str[0])
    df.edited_code1 = df.dx_code.str.split('.').str[1].str[0]
    df.edited_code1.fillna(value='', inplace=True)
    df.dx_code = df.apply(lambda x: x.edited_code if x.edited_code1 == '' else '%s.%s' % (x.edited_code, x.edited_code1),axis=1)
    df.drop(columns=['edited_code', 'edited_code1'], axis=1, inplace=True)
edit_codes(diags_grouped)

In [623]:
print("\n----------- Minimum -----------\n")
print(diags[['dx_date']].min())
 
print("\n----------- Maximum -----------\n")
print(diags[['dx_date']].max())


----------- Minimum -----------

dx_date   1997-05-18
dtype: datetime64[ns]

----------- Maximum -----------

dx_date   2021-08-03
dtype: datetime64[ns]


In [36]:
codes = diags.dx_code.unique()
codes_grouped = diags_grouped.dx_code.unique()

In [37]:
print('from', len(codes), 'to', len(codes_grouped))
del codes, codes_grouped

from 30408 to 12229


Reduced total number of codes from 30408 to 12229

In [1453]:
with open(r'./pickles/diags.pkl', 'wb') as handle:
    pickle.dump(diags, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(r'./pickles/diags_grouped.pkl', 'wb') as handle:
    pickle.dump(diags_grouped, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
with open(r'./pickles/diags.pkl', 'rb') as handle:
    diags = pickle.load(handle)
with open(r'./pickles/diags_grouped.pkl', 'rb') as handle:
    diags_grouped = pickle.load(handle)

#### C. Complication time

In [12]:
no_comps_df = diags[(diags.dx_code == "E11.9") | (diags.dx_code == '250.00')] # create df with entries of uncomplicated diabetes codes only
patient_nums = pd.Series(no_comps_df.patient_num).unique() # list unique patient nums for patients that have e11.9 or 250.00 code
no_comps_sorted = no_comps_df.sort_values(by=['patient_num', 'dx_date']) # sort no_comps_df by patient number and then by dx date

In [14]:
# grab initial diabetes diagnosis date for each patient
def init_diagnoses(patient_nums, df):
    init_comps = {}
    init_diab = {}
    for pt in patient_nums:
        pt_index = df.patient_num.searchsorted(pt, side='left')  # find the first instance on the sorted list where pt appears
        # Get initial diabetes diagnosis time for both dictionaries
        init_comps[pt] = {'initial_DIAB':df.iloc[pt_index].dx_date}
        init_diab[pt] = df.iloc[pt_index].dx_date
    return init_comps, init_diab

# append the time that each patient first experienced a complication to dictionary created above
def get_first_comp(init_comps_dict, comp_name, df):
    temp_df = df[(df.dx_code == comp_name)] # get subset dataframe of specific comp codes
    lst = list(temp_df.patient_num) # generate list of patient ids that have complication
    comp = 'initial_%s' % comp_name
    patient_ids = init_comps_dict.keys() # generate full list of patient numbers
    for init in patient_ids: # loop through pt numbers
        if init in lst:
            temp_comp_time = temp_df[(temp_df.patient_num == init)].iloc[0].dx_date # take the first time a pt experienced a complication
            init_comps_dict[init][comp] = temp_comp_time # ad initial diagnosis date to nested dict
        else:
            init_comps_dict[init][comp] = np.nan # fill with NaN if no complication occured

In [15]:
init_comps_dict, init_diab_diag = init_diagnoses(patient_nums=patient_nums, df=no_comps_sorted)

In [19]:
for comp in comp_names:
    get_first_comp(init_comps_dict, comp, diags)

In [18]:
with open(r'/Users/amomenzadeh/Desktop/DM_pickles/init_diab_diag.pkl', 'wb') as handle:
    pickle.dump(init_diab_diag, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
with open(r'/Users/amomenzadeh/Desktop/DM_pickles/init_comps_dict.pkl', 'wb') as handle:
    pickle.dump(init_comps_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
with open(r'/Users/amomenzadeh/Desktop/DM_pickles/init_diab_diag.pkl', 'rb') as handle:
    init_diab_diag = pickle.load(handle)
with open(r'/Users/amomenzadeh/Desktop/DM_pickles/init_comps_dict.pkl', 'rb') as handle:
    init_comps_dict = pickle.load(handle)

In [13]:
# create new df from  above dictionary for time of first complication with the patient # as the index
comps_time_df = pd.DataFrame.from_dict(init_comps_dict, orient='index')
comps_time_df = comps_time_df.apply(pd.to_datetime, errors='coerce')

# sort table by patient # (index)
comps_time_df.sort_index(inplace=True)

In [14]:
# append columns to df for the time to complications from initial diabetes diagnosis
def get_diffs(comp_names, diff_list, comps_time_df):
    for name in comp_names:
        new_col = '%s_diff' % name
        diff_list.append(new_col)
        temp_comp = 'initial_%s' % name
        comps_time_df[new_col] = (comps_time_df[temp_comp] - comps_time_df['initial_DIAB'])/np.timedelta64(1,"Y")

In [17]:
diff_list = []
get_diffs(comp_names, diff_list, comps_time_df)

In [18]:
# get df of just the times to complications
comps_diff_df = comps_time_df[[i for i in diff_list]]

In [53]:
with open(r'.\pickles\comps_diff_df.pkl', 'wb') as handle:
    pickle.dump(comps_diff_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
with open(r'./pickles/comps_diff_df.pkl', 'rb') as handle:
    comps_diff_df = pickle.load(handle)

In [23]:
# set comps that were diagnosed <1mo from 1st uncomplicated T2DM dx to NaN
comps_over1mo_df=comps_diff_df.mask(comps_diff_df<=(1/12),inplace=False)

In [24]:
comps_over1mo_dropnas=comps_over1mo_df.dropna(axis = 0, how = 'all')

In [25]:
comps_over1mo_dropnas.shape

(21850, 4)

In [26]:
len(comps_over1mo_dropnas.index.unique())

21850

In [27]:
with open(r'/Users/amomenzadeh/Desktop/DM_pickles/comps_over1mo_df.pkl', 'wb') as handle:
    pickle.dump(comps_over1mo_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
with open(r'.\pickles\comps_over1mo_df.pkl', 'rb') as handle:
    comps_over1mo_df = pickle.load(handle)

#### D. Removing entries with codes after inital diab diagnosis

##### Run on grouped codes only (did better than ungrouped in prelim analysis)

In [10]:
#drop the complications prior to first DM code but KEEP DAY OF first DM code
def drop_after_init(df, init_dict):
    drop_list = []
    patient_ids = init_dict.keys()
    for pt in patient_ids:
        drop_list.extend(list(df[(df.patient_num == pt) & 
                                 (df.dx_date > init_dict[pt])].index))
    print('Number to be dropped:', len(drop_list))
    print('Number to keep:', df.shape[0] - len(drop_list))
    df.drop(drop_list, axis=0, inplace=True)
    print('Final DF Shape:', df.shape)

In [73]:
diags_g_prediab = diags_grouped.copy()

NameError: name 'diags_phen' is not defined

In [1451]:
len(diags_g_prediab.patient_num.unique())

30854

In [ ]:
with open(r'./pickles/diags_phen.pickle', 'wb') as handle:
    pickle.dump(diags_phen, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [90]:
with open(r'./pickles/diags_phen.pickle', 'rb') as handle:
    diags_phen = pickle.load(handle)

### 4. Clean Demos input data

In [51]:
demos_c=demos.copy()

In [52]:
len(demos_c.patient_num.unique())

30854

In [53]:
drop = ['sex_c', 'research_contact_pref', 'marital_status_c','language_code',
                       'employment_status_c', 'race_c', 'hispanic_c','need_interpreter', 'ethnicity',
                       'state_c', 'state_name', 'zip3', 'ped_gest_age','primary_care_provider_id', 
                       'primary_care_provider_name','primary_care_provider_title', 'death_date_precision',
                       'vital_status_source', 'death_date_source']
demos_c.drop(columns=drop, axis=1, inplace=True)

In [54]:
# convert birthdate/deathdate to a datetime variable
demos_c.birth_date_shifted = pd.to_datetime(demos.birth_date_shifted, format = '%Y-%m-%d')
demos_c.death_date_shifted = pd.to_datetime(demos.death_date_shifted, format = '%Y-%m-%d')

In [55]:
# add new column with patient initial age at diagnosis
demos_c['init_diab']= demos_c['patient_num'].map(init_diab_diag)

In [56]:
demos_c['age_at_diab'] = (demos_c.init_diab - demos_c.birth_date_shifted)/np.timedelta64(1,'Y')

In [58]:
demos_alive=demos_c.drop(demos_c.loc[demos_c['vital_status']== 'Deceased'].index)

In [59]:
demos_alive.drop(['vital_status','death_date_shifted','birth_date_shifted'],inplace=True, axis=1)

In [60]:
demos_alive['vital_status']='Alive'

In [61]:
demos_alive['duration_diab'] = (datetime.now() - demos_alive.init_diab)/np.timedelta64(1,'Y')

In [62]:
demos_alive.drop(['init_diab'],axis=1,inplace=True)

In [63]:
demo_dec=demos_c[(demos_c.vital_status=='Deceased')]

In [64]:
demo_dec['duration_diab'] = (demo_dec.death_date_shifted - demo_dec.init_diab)/np.timedelta64(1,'Y')


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [67]:
demo_dec.drop(['init_diab','birth_date_shifted','death_date_shifted'],axis=1,inplace=True)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [68]:
all_dfs = [demos_alive, demo_dec]

demos_all=pd.concat(all_dfs).reset_index(drop=True)

In [1960]:
with open(r'./pickles/demos_all.pkl', 'wb') as handle:
    pickle.dump(demos_all, handle, protocol=pickle.HIGHEST_PROTOCOL)    

In [1970]:
demos_all.columns

Index(['patient_num', 'sex', 'marital_status', 'employment_status', 'race',
       'language', 'age_at_diab', 'vital_status', 'duration_diab'],
      dtype='object')

In [1968]:
# % missingness in each column
cols=demos_all.columns
for x in cols:  
    percent_missing = (demos_all[x].isna().sum().sum()) / len(demos_all) *100
    print(x)
    print(percent_missing)  

patient_num
0.0
sex
0.0
marital_status
0.042133921047514095
employment_status
9.40882867699488
race
1.4779283075128022
language
2.4340442082063913
age_at_diab
0.0
vital_status
0.0
duration_diab
0.22363388863680558


In [1969]:
demos_all.to_csv('demos_all.csv')

In [28]:
#Impute using missforest in R - missForest does not work on cat variables in python
demos_R = pd.read_csv(r'/Users/amomenzadeh/Desktop/DM_csv/demos_from_R.csv',low_memory=False)

In [1973]:
# % missingness in each column
cols=demos_R.columns
for x in cols:  
    percent_missing = (demos_R[x].isna().sum().sum()) / len(demos_R) *100
    print(x)
    print(percent_missing)  

sex
0.0
marital_status
0.0
employment_status
0.0
race
0.0
age_at_diab
0.0
vital_status
0.0
duration_diab
0.0


In [1976]:
demos_R['patient_num'] = demos_all['patient_num'].values

In [1986]:
demos_R=demos_R.set_index('patient_num')

In [1990]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)

In [1996]:
# one-hot encode categorical variables
features_to_encode = ['sex', 'marital_status', 'employment_status','race', 'vital_status']
for feature in features_to_encode:
    demos_R = encode_and_bind(demos_R, feature)

In [2002]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [2003]:
# normalize continuous variables
features_to_norm = ['age_at_diab','duration_diab']
x = demos_R[features_to_norm].values
x_scaled = scaler.fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=features_to_norm, index = demos_R.index)
demos_R[features_to_norm] = df_temp

In [2171]:
demos_R.columns

Index(['age_at_diab', 'duration_diab', 'sex_Female', 'sex_Male',
       'marital_status_Divorced', 'marital_status_Legally Separated',
       'marital_status_Married', 'marital_status_Patient Refused',
       'marital_status_Significant Other', 'marital_status_Single',
       'marital_status_Unknown', 'marital_status_Widowed',
       'employment_status_Disabled', 'employment_status_Full Time',
       'employment_status_Not Employed',
       'employment_status_On Active Military Duty',
       'employment_status_Part Time', 'employment_status_Patient Refused',
       'employment_status_Retired', 'employment_status_Self Employed',
       'employment_status_Student - Full Time',
       'employment_status_Student - Part Time', 'employment_status_Unknown',
       'race_American Indian or Alaska Native', 'race_Asian',
       'race_Black or African American', 'race_Multiracial',
       'race_Native Hawaiian or Other Pacific Islander', 'race_Other',
       'race_Unknown', 'race_White or Caucasi

In [2009]:
with open(r'./pickles/demos_R.pkl', 'wb') as handle:
    pickle.dump(demos_R, handle, protocol=pickle.HIGHEST_PROTOCOL)   

In [21]:
with open(r'./pickles/demos_R.pkl', 'rb') as handle:
    demos_R = pickle.load(handle)

### 5. Clean Vitals input data

In [1092]:
vitals_c=vitals.copy()

In [1093]:
len(vitals_c.patient_num.unique())

29746

In [436]:
vitals_c.drop(['height','weight', 'encounter_num'],axis=1,inplace=True)

In [437]:
# remove hours/minutes/seconds places from the date column (leaving only Y-m-d)
temp1 = vitals_c["measure_date_shifted"].tolist()
temp2 = []
for value in temp1:
    x = value.split(" ")
    temp2.append(x[0])
vitals_c["measure_date_shifted"] = temp2

In [438]:
vitals_c.measure_date_shifted = pd.to_datetime(vitals_c.measure_date_shifted, format = '%Y-%m-%d')

In [439]:
vitals_c.rename({'measure_date_shifted':'measure_date'}, axis='columns', inplace=True)
vitals_c.sort_values(by='measure_date', inplace=True) # sort df by date

In [440]:
#include vitals on day of DM dx and before 
def drop_vitals_after_DM(df, init_dict):
    drop_list = []
    patient_ids = init_dict.keys()
    for pt in patient_ids:
        drop_list.extend(list(df[(df.patient_num == pt) & 
                                 (df.measure_date > init_dict[pt])].index))
    print('Number to be dropped:', len(drop_list))
    print('Number to keep:', df.shape[0] - len(drop_list))
    df.drop(drop_list, axis=0, inplace=True)
    print('Final DF Shape:', df.shape)

In [442]:
drop_vitals_after_DM(vitals_c,init_diab_diag)

Number to be dropped: 1136139
Number to keep: 97199
Final DF Shape: (97199, 8)


In [443]:
#number of unique pt IDs with vitals on day of and before first DM dx
len(vitals_c.patient_num.unique())

22475

In [447]:
vitals_ind=vitals_c.set_index('patient_num')

In [448]:
cols = ["bmi","bp_diastolic","bp_systolic","pulse","temperature","respirations"]
vitals_ind[cols] = vitals_ind[cols].replace(['0', 0], np.nan)

In [449]:
vitals_ind['bmi'].values[vitals_ind['bmi'].values <10] = np.nan

In [450]:
vitals_ind['bmi'].values[vitals_ind['bmi'].values >100] = np.nan

In [451]:
cols=vitals_ind.columns
for x in cols[1:]:  
    col_mean = vitals_ind[x].mean()
    col_std = vitals_ind[x].std()
    print(x)
    print ('4 SD range: %s to %s' % (np.round(col_mean-3*col_std), 
                                    np.round(col_mean+3*col_std)) )
    vitals_ind[x].values[vitals_ind[x].values < np.round(col_mean-3*col_std)] = np.nan
    vitals_ind[x].values[vitals_ind[x].values > np.round(col_mean+3*col_std)] = np.nan

bmi
4 SD range: 8.0 to 61.0
bp_diastolic
4 SD range: 45.0 to 110.0
bp_systolic
4 SD range: 77.0 to 187.0
pulse
4 SD range: 38.0 to 122.0
temperature
4 SD range: 95.0 to 101.0
respirations
4 SD range: 8.0 to 27.0


In [452]:
cols=vitals_ind.columns
for x in cols[1:]:  
    print(x)
    print('Range: %s to %s' % (vitals_ind[x].min(),vitals_ind[x].max()))

bmi
Range: 10.73 to 61.0
bp_diastolic
Range: 45.0 to 110.0
bp_systolic
Range: 77.0 to 187.0
pulse
Range: 38.0 to 122.0
temperature
Range: 95.0 to 101.0
respirations
Range: 8.0 to 27.0


In [453]:
cols=vitals_ind.columns
for x in cols:  
    percent_missing = (vitals_ind[x].isna().sum().sum()) / len(vitals_ind) *100
    print(x)
    print(percent_missing)  

measure_date
0.0
bmi
41.29466352534491
bp_diastolic
7.2768238356361685
bp_systolic
7.614275867035669
pulse
14.027922097963971
temperature
42.11360199179004
respirations
49.18980647949053


In [454]:
vitals_ind.drop(['measure_date'],axis=1,inplace=True)

In [455]:
imputer = MissForest()
vitals_MFimputed = imputer.fit_transform(vitals_ind)

Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.


Iteration: 0


Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.


Iteration: 1


Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.


...

Iteration: 5


Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.


Iteration: 6


Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.


Iteration: 7


In [456]:
with open(r'./pickles/vitals_MFimputed.pkl', 'wb') as handle:
    pickle.dump(vitals_MFimputed, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [457]:
imputed_vitals_df = pd.DataFrame(vitals_MFimputed, columns=["bmi",'bp_diastolic','bp_systolic','pulse','temperature','respiration'])

In [458]:
imputed_vitals_df['patient_num'] = vitals_c['patient_num'].values

In [459]:
imputed_vitals_df['measure_date'] = vitals_c['measure_date'].values

In [463]:
imputed_vitals_sorted=imputed_vitals_df.sort_values(by=['patient_num','measure_date'])

In [464]:
imputed_vitals_sort_resetind=imputed_vitals_sorted.reset_index(drop=True)

In [465]:
patient_nums_uniq = pd.Series(imputed_vitals_df.patient_num).unique()

In [468]:
# grab vitals before or on DM dx date for each patient
def last_vitals(patient_nums, df):
    last_vitals_df = pd.DataFrame()
    for pt in patient_nums:
        last_vitals_df=last_vitals_df.append(df[(df.patient_num==pt)].iloc[-1:])
    return last_vitals_df

In [469]:
last_vitals_df=last_vitals(patient_nums=patient_nums_uniq,df=imputed_vitals_sort_resetind)

In [472]:
last_vitals_df=last_vitals_df.set_index('patient_num')

In [473]:
last_vitals_df.isnull().any().any()

False

In [474]:
vitals_df=last_vitals_df.drop(['measure_date'],axis=1)

In [475]:
with open(r'./pickles/vitals_df.pkl', 'wb') as handle:
    pickle.dump(vitals_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [476]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
# normalize 
vitals_norm = pd.DataFrame(scaler.fit_transform(vitals_df), index=vitals_df.index, columns=vitals_df.columns)

In [652]:
vitals_norm.shape

(22475, 6)

In [479]:
with open(r'./pickles/vitals_norm.pkl', 'wb') as handle:
    pickle.dump(vitals_norm, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [651]:
with open(r'./pickles/vitals_norm.pkl', 'rb') as handle:
    vitals_norm = pickle.load(handle)

### 6. Clean Social input data

In [32]:
social_c=social.copy()

In [33]:
len(social_c.patient_num.unique())

29598

In [34]:
social_x=social_c.drop(['encounter_num', 'unknown_fam_hx_yn','edu_level_c','edu_level_name','sexually_active_c', 'sexually_active_name',\
                      'female_partner_yn','male_partner_yn','alcohol_oz_per_wk','alcohol_freq_c','alcohol_freq_name',\
                      'condom_yn', 'pill_yn', 'diaphragm_yn', 'iud_yn', 'surgical_yn', 'spermicide_yn', 'alcohol_drinks_per_day_c',\
                      'implant_yn', 'rhythm_yn', 'injection_yn', 'sponge_yn', 'inserts_yn', 'abstinence_yn','alcohol_drinks_per_day_name',\
                      'fin_resource_strain_c', 'fin_resource_strain_name', 'ipv_emotional_abuse_c', 'ipv_emotional_abuse_name','smoking_quit_date_shifted',\
                      'ipv_fear_c', 'ipv_fear_name', 'ipv_sexual_abuse_c', 'ipv_sexual_abuse_name', 'ipv_physical_abuse_c','ipv_physical_abuse_name',\
                      'living_w_spouse_c', 'living_w_spouse_name', 'daily_stress_c', 'daily_stress_name', 'phone_communication_c', 'phone_communication_name',\
                      'socialization_freq_c', 'socialization_freq_name', 'church_attendance_c', 'church_attendance_name', 'clubmtg_attendance_c',\
                      'clubmtg_attendance_name', 'club_member_c', 'club_member_name', 'phys_act_days_per_week_c', 'phys_act_days_per_week_name',\
                      'phys_act_min_per_sess_c', 'phys_act_min_per_sess_name', 'food_insecurity_scarce_c', 'food_insecurity_scarce_name',\
                      'food_insecurity_worry_c', 'food_insecurity_worry_name', 'med_transport_needs_c', 'med_transport_needs_name',\
                      'other_transport_needs_c', 'other_transport_needs_name','tobacco_pak_per_dy','tobacco_used_years', 'alcohol_use_c',\
                      'ill_drug_user_c','tobacco_user_c','smokeless_tob_use_c'],axis=1)

In [35]:
social_x.isna().any(axis=1).sum()/len(social_x) *100

2.376480302308974

In [36]:
# % missingness in each column - <50% dont need to drop any cols
cols=social_x.columns
for x in cols:  
    percent_missing = (social_x[x].isna().sum().sum()) / len(social_x) *100
    print(x)
    print(percent_missing)  

patient_num
0.0
contact_date_shifted
0.0
alcohol_use_name
1.7583239109735778
ill_drug_user_name
2.315482920733159
tobacco_user_name
1.0179867888597953
cigarettes_yn
0.0
pipes_yn
0.0
cigars_yn
0.0
snuff_yn
0.0
chew_yn
0.0
smokeless_tob_use_name
0.0


In [37]:
# remove hours/minutes/seconds places from the date column (leaving only Y-m-d)
temp1 = social_x["contact_date_shifted"].tolist()
temp2 = []
for value in temp1:
    x = value.split(" ")
    temp2.append(x[0])
social_x["contact_date_shifted"] = temp2

In [38]:
social_x.contact_date_shifted = pd.to_datetime(social_x.contact_date_shifted, format = '%Y-%m-%d')

In [39]:
social_x.rename({'contact_date_shifted':'measure_date'}, axis='columns', inplace=True) # renaming the diagnosis date column
social_x.sort_values(by='measure_date', inplace=True) # sort df by date

In [40]:
#include social hx before and day of DM dx
def drop_social_after_DM(df, init_dict):
    drop_list = []
    patient_ids = init_dict.keys()
    for pt in patient_ids:
        drop_list.extend(list(df[(df.patient_num == pt) & 
                                 (df.measure_date > init_dict[pt])].index))
    print('Number to be dropped:', len(drop_list))
    print('Number to keep:', df.shape[0] - len(drop_list))
    df.drop(drop_list, axis=0, inplace=True)
    print('Final DF Shape:', df.shape)

In [41]:
drop_social_after_DM(social_x,init_diab_diag)

Number to be dropped: 1012767
Number to keep: 62689
Final DF Shape: (62689, 11)


In [42]:
#number of unique pt IDs with social hx on day of and before first DM dx
len(social_x.patient_num.unique())

22184

In [43]:
social_imp=social_x.drop(['patient_num','measure_date'],axis=1)

In [44]:
social_imp.to_csv('social_imp.csv')

In [45]:
social_imp.isnull().sum()

alcohol_use_name           8533
ill_drug_user_name        10191
tobacco_user_name          5698
cigarettes_yn                 0
pipes_yn                      0
cigars_yn                     0
snuff_yn                      0
chew_yn                       0
smokeless_tob_use_name        0
dtype: int64

In [29]:
#Impute using missforest in R - missForest does not work on cat variables in python
social_R = pd.read_csv(r'/Users/amomenzadeh/Desktop/DM_csv/social_imp_from_R.csv',low_memory=False)

In [46]:
social_R['patient_num'] = social_x['patient_num'].values

In [47]:
social_R['measure_date'] = social_x['measure_date'].values

In [48]:
imputed_socials_sorted=social_R.sort_values(by=['patient_num','measure_date'])

In [49]:
imputed_socials_sort_resetind=imputed_socials_sorted.reset_index(drop=True)

In [50]:
patient_nums_uniq = pd.Series(social_R.patient_num).unique()

In [51]:
def last_social(patient_nums, df):
    last_social_df = pd.DataFrame()
    for pt in patient_nums:
        last_social_df=last_social_df.append(df[(df.patient_num==pt)].iloc[-1:])
    return last_social_df

In [52]:
last_social_df=last_social(patient_nums=patient_nums_uniq,df=imputed_socials_sort_resetind)

In [53]:
last_social_df=last_social_df.set_index('patient_num')

In [54]:
last_social_df.isnull().any().any()

False

In [55]:
social_df=last_social_df.drop(['measure_date'],axis=1)

In [56]:
social_df_onehot=social_df.copy()

In [57]:
#one-hot encode
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)

In [58]:
features_to_encode = ['alcohol_use_name', 'ill_drug_user_name', 'tobacco_user_name','cigarettes_yn', 'pipes_yn', 
                      'cigars_yn', 'snuff_yn', 'chew_yn','smokeless_tob_use_name']
for feature in features_to_encode:
    social_df_onehot = encode_and_bind(social_df_onehot, feature)

In [59]:
social_df_onehot.columns

Index(['alcohol_use_name_Never', 'alcohol_use_name_No',
       'alcohol_use_name_Not Asked', 'alcohol_use_name_Not Currently',
       'alcohol_use_name_Yes', 'ill_drug_user_name_Never',
       'ill_drug_user_name_No', 'ill_drug_user_name_Not Asked',
       'ill_drug_user_name_Not Currently', 'ill_drug_user_name_Yes',
       'tobacco_user_name_Never', 'tobacco_user_name_Not Asked',
       'tobacco_user_name_Passive', 'tobacco_user_name_Quit',
       'tobacco_user_name_Yes', 'cigarettes_yn_N', 'cigarettes_yn_Y',
       'pipes_yn_N', 'pipes_yn_Y', 'cigars_yn_N', 'cigars_yn_Y', 'snuff_yn_N',
       'snuff_yn_Y', 'chew_yn_N', 'chew_yn_Y',
       'smokeless_tob_use_name_Current User',
       'smokeless_tob_use_name_Former User',
       'smokeless_tob_use_name_Never Used', 'smokeless_tob_use_name_Unknown'],
      dtype='object')

In [62]:
with open(r'./pickles/social_df.pkl', 'wb') as handle:
    pickle.dump(social_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [60]:
with open(r'/Users/amomenzadeh/Desktop/DM_pickles/social_df_onehot.pkl', 'wb') as handle:
    pickle.dump(social_df_onehot, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(r'./pickles/social_df.pkl', 'rb') as handle:
    social_df = pickle.load(handle)
with open(r'./pickles/social_df_onehot.pkl', 'rb') as handle:
    social_df_onehot = pickle.load(handle)

### 7. Clean Labs

In [69]:
labs_c=labs.copy()

In [70]:
labs_c.columns

Index(['patient_num', 'encounter_num', 'i2b2_date_shifted', 'loinc_code',
       'component_id', 'component_name', 'res_quant_or_qual', 'abnormal_ind',
       'result_num', 'result_unit', 'result_modifier', 'src_value', 'src_unit',
       'order_proc_id', 'proc_id', 'px', 'px_type', 'procedure_name',
       'provider_id', 'provider_name', 'provider_title',
       'specimen_date_shifted', 'result_date_shifted', 'norm_range_low',
       'norm_modifier_low', 'norm_range_high', 'norm_modifier_high'],
      dtype='object')

In [71]:
len(labs_c.patient_num.unique())

29579

In [74]:
labs_c.drop(['loinc_code','component_id','res_quant_or_qual','abnormal_ind','px','px_type','procedure_name',
             'provider_id','provider_name','provider_title','specimen_date_shifted','result_date_shifted',
             'norm_range_low', 'norm_modifier_low','norm_range_high','norm_modifier_high'],inplace=True, axis=1)

In [75]:
labs_c.columns

Index(['patient_num', 'encounter_num', 'i2b2_date_shifted', 'component_name',
       'result_num', 'result_unit', 'result_modifier', 'src_value', 'src_unit',
       'order_proc_id', 'proc_id'],
      dtype='object')

In [1040]:
# remove hours/minutes/seconds places from the date column (leaving only Y-m-d)
temp1 = labs_c["i2b2_date_shifted"].tolist()
temp2 = []
for value in temp1:
    x = value.split(" ")
    temp2.append(x[0])
labs_c["i2b2_date_shifted"] = temp2

In [1041]:
labs_c.i2b2_date_shifted = pd.to_datetime(labs_c.i2b2_date_shifted, format = '%Y-%m-%d')

In [1042]:
labs_c.rename({'i2b2_date_shifted':'measure_date'}, axis='columns', inplace=True) # renaming the diagnosis date column
labs_c.sort_values(by='measure_date', inplace=True) # sort df by date

In [1043]:
#include labs only on day of DM dx and before 
def drop_labs_after_DM(df, init_dict):
    drop_list = []
    patient_ids = init_dict.keys()
    for pt in patient_ids:
        drop_list.extend(list(df[(df.patient_num == pt) & 
                                 (df.measure_date > init_dict[pt])].index))
    print('Number to be dropped:', len(drop_list))
    print('Number to keep:', df.shape[0] - len(drop_list))
    df.drop(drop_list, axis=0, inplace=True)
    print('Final DF Shape:', df.shape)

In [1045]:
drop_labs_after_DM(labs_c,init_diab_diag)

Number to be dropped: 32461813
Number to keep: 2483846
Final DF Shape: (2483846, 11)


In [1049]:
with open(r'.\pickles\labs_c.pkl', 'wb') as handle:
    pickle.dump(labs_c, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(r'./pickles/labs_c.pkl', 'rb') as handle:
    labs_c = pickle.load(handle)

In [1046]:
#number of unique pt IDs with labs on day of and before first DM dx
len(labs_c.patient_num.unique())

21293

In [1059]:
labs_prior=labs_c.drop(['encounter_num','result_modifier','src_value','src_unit','order_proc_id','proc_id'],axis=1)

In [1127]:
#Take 100 most common labs
col = 'component_name'
n = 4800

labs_freq=labs_prior[labs_prior.groupby(col)[col].transform('count').ge(n)]

In [1151]:
labs_freq.component_name.value_counts()

CREATININE         58526
GLUCOSE            57152
POTASSIUM          56412
SODIUM             53321
CALCIUM            53318
                   ...  
URINE MUCUS         5592
BUN/CREAT RATIO     5151
CREATINE KINASE     5098
PRE                 4865
TROPONIN I          4834
Name: component_name, Length: 100, dtype: int64

In [1212]:
#Take labs closest to T2DM dx date
def last_labs(patient_nums, df):
    last_labs_dict = {}
    for pt in patient_nums:
        last_labs_dict[pt]={}
    ##outer loop begins
    components=df.component_name.unique()
    for x in components:
        temp_df=df[(df.component_name==x)].sort_values(by='measure_date')


        for pt in patient_nums:
            #print('starting 2nd loop')
            #print(components[0])
            #print(pt)
            if len(temp_df[(temp_df.patient_num==pt)].iloc[-1:])==0:
                last_labs_dict[pt][x] = np.nan
            else:
                last_labs_dict[pt][x]=temp_df[(temp_df.patient_num==pt)].iloc[-1:].result_num.values[0]
    return last_labs_dict

In [1140]:
patient_nums_uniq = pd.Series(labs_freq.patient_num).unique()

In [1179]:
len(patient_nums_uniq)

20221

In [ ]:
last_labs_dict=last_labs(patient_nums=patient_nums_uniq,df=labs_freq)

In [1214]:
last_labs_df=pd.DataFrame.from_dict(last_labs_dict, orient='index')

In [1784]:
with open(r'./pickles/last_labs_df.pkl', 'wb') as handle:
    pickle.dump(last_labs_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [488]:
with open(r'./pickles/last_labs_df.pkl', 'rb') as handle:
    last_labs_df = pickle.load(handle)

In [1254]:
# % missingness in each column
cols=last_labs_df.columns
for x in cols:  
    percent_missing = (last_labs_df[x].isna().sum().sum()) / len(last_labs_df) *100
    print(x)
    print(percent_missing)  

CHOLESTEROL LDL
81.81098857623263
TOTAL CHOLESTEROL/HDL RATIO
68.1914841006874
CHOLESTEROL
65.08580188912518
TRIGLYCERIDES
64.67039216655952
CHOLESTEROL HDL
65.47153948865041
PLATELET COUNT
29.65234162504327
AST/SGOT
47.33692695712378
CALCIUM
18.579694377132682
GLYCOHEMOGLOBIN
53.59279956480886
GLUCOSE
16.329558379902082
BICARBONATE
34.82023638791355
CHLORIDE
18.668710746253893
POTASSIUM
18.4115523465704
HEMATOCRIT
29.058899164235203
WHITE BLOOD CELL COUNT
29.780920824885023
CREATININE
16.631224964146185
TOTAL PROTEIN
48.8749320013847
ALBUMIN
48.06883932545374
BLOOD UREA NITROGEN
18.752781761535037
INTERNATIONAL NORMALIZED RATIO
71.80653775777657
PROTIME
76.79145442856436
PHOSPHORUS
86.17279066317195
ANION GAP
33.40586518965432
TOTAL CO2
79.6251421789229
EGFR
31.442559715147617
EGFR FOR AFRICAN AMERICANS
37.47589140002967
SODIUM
18.51540477721181
BILIRUBIN TOTAL
48.306216309776964
ALKALINE PHOSPHATASE
48.19247317145542
ALT/SGPT
46.03135354334603
URIC ACID
91.90940111764996
HEMOGLOBIN
2

In [493]:
codes_demos_vit_soc_norm.shape

(19528, 8632)

In [514]:
#match labs to other inputs combined to limit # missingness
labs_matched = last_labs_df.merge(codes_demos_vit_soc_norm,left_index=True, right_index=True)

In [515]:
len(labs_matched)

15987

In [516]:
labs_matched=labs_matched.iloc[:,:-8632]

In [518]:
with open(r'./pickles/labs_matched.pkl', 'wb') as handle:
    pickle.dump(labs_matched, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [519]:
#Drop in place cols with >=50% missingness
cols=labs_matched.columns
for x in cols:  
    percent_missing = (labs_matched[x].isna().sum()) / len(labs_matched) *100
    print(x)
    print(percent_missing)
    if percent_missing >= 50:
        labs_matched.drop([x],axis=1, inplace=True)

CHOLESTEROL LDL
82.51704509914305
TOTAL CHOLESTEROL/HDL RATIO
70.56358291111529
CHOLESTEROL
68.16788640770626
TRIGLYCERIDES
67.77381622568338
CHOLESTEROL HDL
68.50566084944018
PLATELET COUNT
25.996121849002314
AST/SGOT
46.25633327078251
CALCIUM
17.852004753862513
GLYCOHEMOGLOBIN
56.64602489522738
GLUCOSE
15.481328579470821
BICARBONATE
30.58735222368174
CHLORIDE
17.708137862012883
POTASSIUM
17.751923437793206
HEMATOCRIT
25.583286420216428
WHITE BLOOD CELL COUNT
26.133733658597613
CREATININE
16.16313254519297
TOTAL PROTEIN
47.31969725401889
ALBUMIN
46.61912804153375
BLOOD UREA NITROGEN
17.80821917808219
INTERNATIONAL NORMALIZED RATIO
69.68161631325452
PROTIME
75.19234377932071
PHOSPHORUS
86.40770626133734
ANION GAP
31.369237505473198
TOTAL CO2
81.99787327203353
EGFR
25.095390004378558
EGFR FOR AFRICAN AMERICANS
28.416838681428665
SODIUM
17.576781134671922
BILIRUBIN TOTAL
46.956902483267655
ALKALINE PHOSPHATASE
46.888096578470005
ALT/SGPT
45.41189716644774
URIC ACID
91.81835241133422
HEMO

In [520]:
labs_matched.count()

PLATELET COUNT                    11831
AST/SGOT                           8592
CALCIUM                           13133
GLUCOSE                           13512
BICARBONATE                       11097
CHLORIDE                          13156
POTASSIUM                         13149
HEMATOCRIT                        11897
WHITE BLOOD CELL COUNT            11809
CREATININE                        13403
TOTAL PROTEIN                      8422
ALBUMIN                            8534
BLOOD UREA NITROGEN               13140
ANION GAP                         10972
EGFR                              11975
EGFR FOR AFRICAN AMERICANS        11444
SODIUM                            13177
BILIRUBIN TOTAL                    8480
ALKALINE PHOSPHATASE               8491
ALT/SGPT                           8727
HEMOGLOBIN                        11909
RED BLOOD CELL COUNT              11768
MEAN CORPUSCULAR HGB              11642
MEAN CORPUSCULAR VOLUME           11664
MEAN PLATELET VOLUME              11429


In [ ]:
labs_matched['CREATININE'].values[labs_matched['CREATININE'].values > 50]= np.nan

In [526]:
cols=labs_matched.columns
labs_matched[cols] = labs_matched[cols].replace(['0', 0], np.nan)

In [527]:
cols=labs_matched.columns
for x in cols:  
    print(x)
    print('min:', labs_matched[x].min())
    print('max:', labs_matched[x].max())

PLATELET COUNT
min: 2.0
max: 1663.0
AST/SGOT
min: 0.1
max: 6764.0
CALCIUM
min: 0.6
max: 18.0
GLUCOSE
min: 4.0
max: 1282.0
BICARBONATE
min: 2.0
max: 98.0
CHLORIDE
min: 69.0
max: 140.0
POTASSIUM
min: 0.78
max: 8.9
HEMATOCRIT
min: 12.0
max: 66.0
WHITE BLOOD CELL COUNT
min: 0.2
max: 19270.0
CREATININE
min: 0.2
max: 10.6
TOTAL PROTEIN
min: 0.3
max: 77.0
ALBUMIN
min: 0.2
max: 4040.0
BLOOD UREA NITROGEN
min: 2.0
max: 211.0
ANION GAP
min: 1.0
max: 78.0
EGFR
min: 4.0
max: 255.0
EGFR FOR AFRICAN AMERICANS
min: 6.0
max: 308.0
SODIUM
min: 102.0
max: 177.0
BILIRUBIN TOTAL
min: 0.1
max: 58.0
ALKALINE PHOSPHATASE
min: 5.0
max: 3003.0
ALT/SGPT
min: 0.7
max: 3826.0
HEMOGLOBIN
min: 3.1
max: 42.5
RED BLOOD CELL COUNT
min: 1.0
max: 8.1
MEAN CORPUSCULAR HGB
min: 12.0
max: 40.4
MEAN CORPUSCULAR VOLUME
min: 31.4
max: 126.8
MEAN PLATELET VOLUME
min: 4.9
max: 16.6
RED CELL DISTRIBUTION WIDTH
min: 10.6
max: 313.0
MEAN CORPUSCULAR HGB CONCENTRN
min: 23.0
max: 45.8
MONOCYTE PERCENT
min: 0.3
max: 100.0
NEUTROPHIL 

In [528]:
cols=labs_matched.columns
for x in cols:  
    col_mean = labs_matched[x].mean()
    col_std = labs_matched[x].std()
    print(x)
    print ('3 SD range: %s to %s' % (np.round(col_mean-3*col_std), 
                                    np.round(col_mean+3*col_std)) )
    labs_matched[x].values[labs_matched[x].values < np.round(col_mean-3*col_std)] = np.nan
    labs_matched[x].values[labs_matched[x].values > np.round(col_mean+3*col_std)] = np.nan

PLATELET COUNT
3 SD range: -35.0 to 541.0
AST/SGOT
3 SD range: -284.0 to 355.0
CALCIUM
3 SD range: 7.0 to 11.0
GLUCOSE
3 SD range: -141.0 to 508.0
BICARBONATE
3 SD range: 14.0 to 36.0
CHLORIDE
3 SD range: 87.0 to 115.0
POTASSIUM
3 SD range: 3.0 to 6.0
HEMATOCRIT
3 SD range: 23.0 to 56.0
WHITE BLOOD CELL COUNT
3 SD range: -3832.0 to 4289.0
CREATININE
3 SD range: -0.0 to 2.0
TOTAL PROTEIN
3 SD range: 4.0 to 11.0
ALBUMIN
3 SD range: -211.0 to 222.0
BLOOD UREA NITROGEN
3 SD range: -13.0 to 48.0
ANION GAP
3 SD range: 0.0 to 24.0
EGFR
3 SD range: 17.0 to 100.0
EGFR FOR AFRICAN AMERICANS
3 SD range: 12.0 to 112.0
SODIUM
3 SD range: 126.0 to 149.0
BILIRUBIN TOTAL
3 SD range: -4.0 to 6.0
ALKALINE PHOSPHATASE
3 SD range: -188.0 to 384.0
ALT/SGPT
3 SD range: -198.0 to 270.0
HEMOGLOBIN
3 SD range: 7.0 to 19.0
RED BLOOD CELL COUNT
3 SD range: 2.0 to 7.0
MEAN CORPUSCULAR HGB
3 SD range: 22.0 to 38.0
MEAN CORPUSCULAR VOLUME
3 SD range: 68.0 to 108.0
MEAN PLATELET VOLUME
3 SD range: 5.0 to 14.0
RED CE

In [529]:
cols=labs_matched.columns
for x in cols:  
    print(x)
    print('Range: %s to %s' % (labs_matched[x].min(),labs_matched[x].max()))

PLATELET COUNT
Range: 2.0 to 540.0
AST/SGOT
Range: 0.1 to 348.0
CALCIUM
Range: 7.0 to 11.0
GLUCOSE
Range: 4.0 to 508.0
BICARBONATE
Range: 14.0 to 36.0
CHLORIDE
Range: 87.0 to 115.0
POTASSIUM
Range: 3.0 to 6.0
HEMATOCRIT
Range: 23.0 to 56.0
WHITE BLOOD CELL COUNT
Range: 0.2 to 4280.0
CREATININE
Range: 0.2 to 2.0
TOTAL PROTEIN
Range: 4.0 to 10.7
ALBUMIN
Range: 0.2 to 17.0
BLOOD UREA NITROGEN
Range: 2.0 to 48.0
ANION GAP
Range: 1.0 to 24.0
EGFR
Range: 17.0 to 100.0
EGFR FOR AFRICAN AMERICANS
Range: 12.0 to 112.0
SODIUM
Range: 126.0 to 149.0
BILIRUBIN TOTAL
Range: 0.1 to 5.9
ALKALINE PHOSPHATASE
Range: 5.0 to 381.0
ALT/SGPT
Range: 0.7 to 261.0
HEMOGLOBIN
Range: 7.0 to 19.0
RED BLOOD CELL COUNT
Range: 2.0 to 6.99
MEAN CORPUSCULAR HGB
Range: 22.0 to 38.0
MEAN CORPUSCULAR VOLUME
Range: 68.0 to 108.0
MEAN PLATELET VOLUME
Range: 5.5 to 14.0
RED CELL DISTRIBUTION WIDTH
Range: 10.6 to 25.0
MEAN CORPUSCULAR HGB CONCENTRN
Range: 29.0 to 38.0
MONOCYTE PERCENT
Range: 0.3 to 16.0
NEUTROPHIL PERCENT
Ra

In [530]:
cols=labs_matched.columns
for x in cols:  
    percent_missing = (labs_matched[x].isna().sum()) / len(labs_matched) *100
    print(x)
    print(percent_missing)  

PLATELET COUNT
26.771752048539437
AST/SGOT
46.46900606742979
CALCIUM
18.583849377619316
GLUCOSE
17.00756864952774
BICARBONATE
31.350472258710205
CHLORIDE
18.47751297929568
POTASSIUM
18.690185775942954
HEMATOCRIT
26.089948082817287
WHITE BLOOD CELL COUNT
28.11659473322074
CREATININE
18.21479952461375
TOTAL PROTEIN
47.382248076562206
ALBUMIN
46.63789328829674
BLOOD UREA NITROGEN
19.16557202727216
ANION GAP
31.963470319634702
EGFR
27.040720585475697
EGFR FOR AFRICAN AMERICANS
30.549821730155752
SODIUM
18.22730968912241
BILIRUBIN TOTAL
47.44479889910552
ALKALINE PHOSPHATASE
47.5386251329205
ALT/SGPT
45.79345718396197
HEMOGLOBIN
25.958591355476322
RED BLOOD CELL COUNT
26.509038593857507
MEAN CORPUSCULAR HGB
28.235441296053043
MEAN CORPUSCULAR VOLUME
27.872646525301807
MEAN PLATELET VOLUME
28.56696065553262
RED CELL DISTRIBUTION WIDTH
27.69750422218052
MEAN CORPUSCULAR HGB CONCENTRN
27.57240257709389
MONOCYTE PERCENT
43.210108212923
NEUTROPHIL PERCENT
43.028710827547386
BASOPHIL PERCENTAGE
6

In [531]:
labs_matched.drop(['BASOPHIL ABSOLUTE', 'BASOPHIL PERCENTAGE'],axis=1,inplace=True)

In [532]:
imputer = MissForest()
labs_MFimputed = imputer.fit_transform(labs_matched)

Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error

Iteration: 0


Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error

Iteration: 1


Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error

Iteration: 2


In [533]:
labs_matched.columns

Index(['PLATELET COUNT', 'AST/SGOT', 'CALCIUM', 'GLUCOSE', 'BICARBONATE',
       'CHLORIDE', 'POTASSIUM', 'HEMATOCRIT', 'WHITE BLOOD CELL COUNT',
       'CREATININE', 'TOTAL PROTEIN', 'ALBUMIN', 'BLOOD UREA NITROGEN',
       'ANION GAP', 'EGFR', 'EGFR FOR AFRICAN AMERICANS', 'SODIUM',
       'BILIRUBIN TOTAL', 'ALKALINE PHOSPHATASE', 'ALT/SGPT', 'HEMOGLOBIN',
       'RED BLOOD CELL COUNT', 'MEAN CORPUSCULAR HGB',
       'MEAN CORPUSCULAR VOLUME', 'MEAN PLATELET VOLUME',
       'RED CELL DISTRIBUTION WIDTH', 'MEAN CORPUSCULAR HGB CONCENTRN',
       'MONOCYTE PERCENT', 'NEUTROPHIL PERCENT', 'EOSINOPHIL PERCENT',
       'LYMPH PERCENT', 'NEUTROPHIL ABSOLUTE', 'LYMPHOCYTE ABSOLUTE',
       'MONOCYTE ABSOLUTE', 'EOSINOPHIL ABSOLUTE'],
      dtype='object')

In [535]:
labs_matched.shape

(15987, 35)

In [534]:
imputed_labs_df = pd.DataFrame(labs_MFimputed, columns=['PLATELET COUNT', 'AST/SGOT', 'CALCIUM', 'GLUCOSE','BICARBONATE', 
                                                        'CHLORIDE', 'POTASSIUM', 'HEMATOCRIT','WHITE BLOOD CELL COUNT', 
                                                        'CREATININE', 'TOTAL PROTEIN', 'ALBUMIN','BLOOD UREA NITROGEN', 
                                                        'ANION GAP', 'EGFR','EGFR FOR AFRICAN AMERICANS', 'SODIUM', 
                                                        'BILIRUBIN TOTAL','ALKALINE PHOSPHATASE', 'ALT/SGPT', 'HEMOGLOBIN',
                                                        'RED BLOOD CELL COUNT', 'MEAN CORPUSCULAR HGB',
                                                        'MEAN CORPUSCULAR VOLUME', 'MEAN PLATELET VOLUME',
                                                        'RED CELL DISTRIBUTION WIDTH', 'MEAN CORPUSCULAR HGB CONCENTRN',
                                                        'MONOCYTE PERCENT', 'NEUTROPHIL PERCENT','EOSINOPHIL PERCENT', 
                                                        'LYMPH PERCENT', 'NEUTROPHIL ABSOLUTE','LYMPHOCYTE ABSOLUTE', 
                                                        'MONOCYTE ABSOLUTE','EOSINOPHIL ABSOLUTE'])

In [536]:
imputed_labs_df.shape

(15987, 35)

In [537]:
imputed_labs_df['patient_num'] = labs_matched.index.values

In [538]:
imp_labs=imputed_labs_df.set_index('patient_num')

In [539]:
len(imputed_labs_df.patient_num.unique()) 

15987

In [544]:
imp_labs.isnull().any().any()

False

In [545]:
with open(r'./pickles/imp_labs.pkl', 'wb') as handle:
    pickle.dump(imp_labs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [1188]:
with open(r'./pickles/imp_labs.pkl', 'rb') as handle:
    imp_labs = pickle.load(handle)

In [ ]:
#normalize
labs_norm = pd.DataFrame(scaler.fit_transform(imp_labs), columns=imp_labs.columns, index=imp_labs.index)

In [547]:
with open(r'./pickles/labs_norm.pkl', 'wb') as handle:
    pickle.dump(labs_norm, handle, protocol=pickle.HIGHEST_PROTOCOL)

### 8. Convert ICD codes to phenotypes

#### A. Convert all codes in diags in phenotypes

In [8]:
with open(r'./pickles/diags.pkl', 'rb') as handle:
    diags = pickle.load(handle)

In [7]:
listofzeros = [0] * len(diags)
len(listofzeros)

37021872

In [ ]:
listofzeros = [0] * len(diags)
i=0
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))
tmplooptime = time.time()
for x in diags.dx_code:
    if x in icd9map.index:
        listofzeros[i] = icd9map.loc[[x]].Phenotype.values[0]
    elif x in icd10map.index:
        listofzeros[i] = icd10map.loc[[x]].Phenotype.values[0]
    elif x == 'OPTH':
        listofzeros[i] = 'OPTH'
    elif x == 'CVD':
        listofzeros[i] = 'CVD'
    elif x == 'KIDNEY':
        listofzeros[i] = 'KIDNEY'
    elif x == 'NEUROPATH':
        listofzeros[i] = 'NEUROPATH'
    else:
        listofzeros[i] = 'NaN'
    if i%100000==0:
        print('+++++++++++++++')
        print(i)
        print('time per 100k:',np.round((time.time()-tmplooptime)/60,2), ' minutes')
        print('total time:',np.round((time.time() - start_time)/60,2), 'minutes')
        tmplooptime = time.time()
    i+=1

--- 6.103515625e-05 seconds ---
+++++++++++++++
0
time per 100k: 0.0  minutes
total time: 0.0 minutes
+++++++++++++++
100000
time per 100k: 0.64  minutes
total time: 0.64 minutes
+++++++++++++++
200000
time per 100k: 0.63  minutes
total time: 1.27 minutes
+++++++++++++++
300000
time per 100k: 0.63  minutes
total time: 1.9 minutes
+++++++++++++++
400000
time per 100k: 0.65  minutes
total time: 2.55 minutes
+++++++++++++++
500000
time per 100k: 0.63  minutes
total time: 3.18 minutes
+++++++++++++++
600000
time per 100k: 0.57  minutes
total time: 3.75 minutes
+++++++++++++++
700000
time per 100k: 0.55  minutes
total time: 4.31 minutes
+++++++++++++++
800000
time per 100k: 0.55  minutes
total time: 4.86 minutes
+++++++++++++++
900000
time per 100k: 0.55  minutes
total time: 5.41 minutes
+++++++++++++++
1000000
time per 100k: 0.63  minutes
total time: 6.04 minutes
+++++++++++++++
1100000
time per 100k: 0.64  minutes
total time: 6.68 minutes
+++++++++++++++
1200000
time per 100k: 0.64  minut

In [ ]:
with open(r'./pickles/listofzeros.pkl', 'wb') as handle:
    pickle.dump(listofzeros, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
diags['Phenotypes'] = listofzeros

In [ ]:
diags_phen=diags.drop(['dx_code'],axis=1)

#### B. Removing phenotypes after inital diab diagnosis

In [9]:
with open(r'./pickles/init_diab_diag.pkl', 'rb') as handle:
    init_diab_diag = pickle.load(handle)
with open(r'./pickles/init_comps_dict.pkl', 'rb') as handle:
    init_comps_dict = pickle.load(handle)

In [10]:
#drop the complications prior to first DM code but KEEP DAY OF first DM code
def drop_after_init(df, init_dict):
    drop_list = []
    patient_ids = init_dict.keys()
    for pt in patient_ids:
        drop_list.extend(list(df[(df.patient_num == pt) & 
                                 (df.dx_date > init_dict[pt])].index))
    print('Number to be dropped:', len(drop_list))
    print('Number to keep:', df.shape[0] - len(drop_list))
    df.drop(drop_list, axis=0, inplace=True)
    print('Final DF Shape:', df.shape)

In [ ]:
drop_after_init(diags_phen, init_diab_diag)

In [ ]:
with open(r'./pickles/diags_phen.pickle', 'wb') as handle:
    pickle.dump(diags_phen, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [64]:
with open(r'./pickles/diags_phen.pickle', 'rb') as handle:
    diags_phen = pickle.load(handle)

### 9. Model inputs

#### A. Create pivot table of prediab phenotypes, binarize, drop NaN col

In [62]:
with open(r'./pickles/demos_R.pkl', 'rb') as handle:
    demos_R = pickle.load(handle)
with open(r'/Users/amomenzadeh/Desktop/DM_pickles/social_df_onehot.pkl', 'rb') as handle:
    social_df_onehot = pickle.load(handle)
with open(r'./pickles/vitals_norm.pkl', 'rb') as handle:
    vitals_norm = pickle.load(handle)
with open(r'./pickles/labs_norm.pkl', 'rb') as handle:
    labs_norm = pickle.load(handle)

In [65]:
#binarize
phen_prediab_bin = diags_phen.reset_index().pivot_table(index='patient_num', columns='Phenotypes', values='index',   # create pivot table of binary entries for dx code before diab diagnosis
                                                              aggfunc='count', fill_value=0).clip(upper=1)

In [66]:
phen_prediab_bin.drop(['NaN'],axis=1,inplace=True)

In [67]:
phen_prediab_bin.shape

(30854, 1721)

In [ ]:
with open(r'./pickles/phen_prediab_bin.pkl', 'wb') as handle:
    pickle.dump(phen_prediab_bin, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### B. Merge Phenotypes + Demos

In [68]:
phen_demos = phen_prediab_bin.merge(demos_R,left_index=True, right_index=True)
phen_demos.shape

(30854, 1754)

#### C. Merge Phenotypes + Demos + Vitals

In [69]:
phen_demos_vit = phen_demos.merge(vitals_norm,left_index=True, right_index=True)
phen_demos_vit.shape

(22475, 1760)

#### D. Merge Phenotypes + Demos + Vitals + Social

In [70]:
phen_demos_vit_soc = phen_demos_vit.merge(social_df_onehot, left_index=True, right_index=True)
phen_demos_vit_soc.shape

(19528, 1789)

#### E. Merge Phenotypes + Demos + Vitals + Social + Labs

In [71]:
phen_demos_vit_soc_lab = phen_demos_vit_soc.merge(labs_norm, left_index=True, right_index=True)
phen_demos_vit_soc_lab.shape

(15987, 1824)

In [72]:
with open(r'/Users/amomenzadeh/Desktop/DM_pickles/phen_demos_vit_soc_lab.pkl', 'wb') as handle:
    pickle.dump(phen_demos_vit_soc_lab, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(r'./pickles/phen_demos_vit_soc_lab.pkl', 'rb') as handle:
    phen_demos_vit_soc_lab = pickle.load(handle) 